In [1]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv

from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [5]:

import random
from datasets import load_dataset, Dataset
import json
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion
import os

domain_actions_schema = {
    "type": "object",
    "properties": {
        "actions": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the action 'create user' or 'place order'",
                    },
                    "description": {
                        "type": "string",
                        "description": "The description of the action, when it should happen and its outcome",
                    },
                    "parameter_schema": {
                        "type": "object",
                        "description": "The detailed list of parameters as valid json-schema. It contains all the information required to correctly execute the action, like names of relevant entities, domain specfic codes, timestamps, notes, etc.",
                        "properties": {
                            "type": {
                                "type": "string"
                            },
                            "description": {
                                "type": "string"
                            },
                            "properties": {
                                "type": "object",
                            },
                        },
                        "additionalProperties": True,
                        "required": ["type", "properties"]
                    },
                    "result_schema": {
                        "type": "object",
                        "description": "The result of the action as valid json-schema. It contains information about the state after the action has been executed.",
                        "properties": {
                            "type": {
                                "type": "string"
                            },
                            "description": {
                                "type": "string"
                            },
                            "properties": {
                                "type": "object",
                            },
                        },
                        "additionalProperties": True,
                        "required": ["type", "properties"],
                    }
                },
                "required": ["name", "description", "parameter_schema", "result_schema"]
            },
            "minItems": 6,
        },
    }
}

def generate_domain_actions(entry):
    results = []

    available_models = [
        "mistralai/mistral-nemo",
        #"mistralai/mixtral-8x7b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-small",
        #"mistralai/mistral-medium",
        "mistralai/mistral-large"
    ]

    model = random.choice(available_models)

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert {entry['profession']}. A domain specific action is a task that you perform in your profession. For example, if you are a doctor, a domain specific action could be 'prescribe medication'."
        }, {
            "role": "user",
            "content": f"Generate a list of 6 domain specific actions for the profession {entry['profession']} in the process {entry['process']}. Use the tool call generate_actions to generate the list of actions."
        }],
        tools=[{
            "type": "function",
            "function": {
                "name": "generate_actions",
                "parameters": domain_actions_schema,
            },
        }],
        tool_choice="required",
        max_tokens=8191,
        temperature=1.2,
        extra_body={
            "provider": {
                "require_parameters": True,
            },
        }
    )

    domain_actions = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)

    for action in domain_actions["actions"]:
        results.append({
            "profession": entry["profession"],
            "process": entry["process"],
            "name": action["name"],
            "description": action["description"],
            "parameter_schema": action["parameter_schema"],
            "result_schema": action["result_schema"],
        })
    
    return results


processes = load_dataset("lucaelin/generic_processes_v1", split="train")
domain_actions = processes.shuffle().map(
    sane_batch_map(generate_domain_actions, {"profession": [], "process": [], "name": [], "description": [], "parameter_schema": [], "result_schema": []}), 
    batched=True, 
    batch_size=4, 
    num_proc=6, 
    remove_columns=processes.column_names
)
domain_actions.sort(["profession", "process", "name"]).to_list()
#domain_actions.sort(["profession", "process", "name"]).push_to_hub("lucaelin/generic_domain_actions_v1")

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

{'profession': 'Executive Assistant', 'process': 'Travel Arrangements'}
Traceback (most recent call last):
  File "/tmp/ipykernel_819413/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_819413/448983896.py", line 125, in generate_domain_actions
    "result_schema": action["result_schema"],
                     ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'result_schema'



Map (num_proc=4):   1%|          | 6/1000 [01:20<3:43:25, 13.49s/ examples]

{'profession': 'Clinical Social Worker', 'process': 'Crisis Intervention'}
Traceback (most recent call last):
  File "/tmp/ipykernel_819413/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_819413/448983896.py", line 125, in generate_domain_actions
    "result_schema": action["result_schema"],
                     ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'result_schema'



Map (num_proc=4):   1%|          | 12/1000 [01:49<2:18:11,  8.39s/ examples]

...

{'profession': 'Physical Therapist', 'process': 'Billing and Coding'}
Traceback (most recent call last):
  File "/tmp/ipykernel_819413/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_819413/448983896.py", line 125, in generate_domain_actions
    "result_schema": action["result_schema"],
                     ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'result_schema'



Map (num_proc=4):  99%|█████████▊| 986/1000 [1:55:09<04:00, 17.18s/ examples]

{'profession': 'Bookkeeping, Accounting, & Audit Clerk', 'process': 'Process Invoices'}
Traceback (most recent call last):
  File "/tmp/ipykernel_819413/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_819413/448983896.py", line 125, in generate_domain_actions
    "result_schema": action["result_schema"],
                     ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'result_schema'



Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_domain_actions_v1/commit/18e27e534c4d27c3bf7406b9565539de766df552', commit_message='Upload dataset', commit_description='', oid='18e27e534c4d27c3bf7406b9565539de766df552', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
from datasets import load_dataset, Dataset
import json
import jsonschema

def fix_schema(entry):
    """
    use jsonschema to validate parameter_schema and result_schema against the json-schema meta-schema, and remove properties that are not valid
    """
    
    try:
        jsonschema.Draft7Validator.check_schema(json.loads(entry["parameter_schema"]))
    except jsonschema.exceptions.SchemaError as e:
        print(f"parameter_schema error: {e}")
        return []
    
    try:
        jsonschema.Draft7Validator.check_schema(json.loads(entry["result_schema"]))
    except jsonschema.exceptions.SchemaError as e:
        return []
    
    return [entry]

  

processes = load_dataset("lucaelin/generic_domain_actions_v1", split="train")
domain_actions = processes.shuffle().map(
    sane_batch_map(fix_schema, {"profession": [], "process": [], "name": [], "description": [], "parameter_schema": [], "result_schema": []}), 
    batched=True, 
    batch_size=4, 
    num_proc=6, 
    remove_columns=processes.column_names
)
#domain_actions.sort(["profession", "process", "name"]).to_list()
domain_actions.sort(["profession", "process", "name"]).push_to_hub("lucaelin/generic_domain_actions_v1")

Map (num_proc=6):   0%|          | 0/4319 [00:00<?, ? examples/s]

parameter_schema error: 'Equipment for maintenance (e.g., vacuum cleaner, brooms).' is not of type 'object', 'boolean'

Failed validating 'type' in metaschema['properties']['properties']['additionalProperties']:
    {'$schema': 'http://json-schema.org/draft-07/schema#',
     '$id': 'http://json-schema.org/draft-07/schema#',
     'title': 'Core schema meta-schema',
     'definitions': {'schemaArray': {'type': 'array',
                                     'minItems': 1,
                                     'items': {'$ref': '#'}},
                     'nonNegativeInteger': {'type': 'integer',
                                            'minimum': 0},
                     'nonNegativeIntegerDefault0': {'allOf': [{'$ref': '#/definitions/nonNegativeInteger'},
                                                              {'default': 0}]},
                     'simpleTypes': {'enum': ['array',
                                              'boolean',
                                           

Map (num_proc=6):   9%|▊         | 368/4319 [00:00<00:01, 3096.32 examples/s]


parameter_schema error: 'The JSON data structure describing the safety training.' is not of type 'object', 'boolean'

Failed validating 'type' in metaschema['properties']['properties']['additionalProperties']:
    {'$schema': 'http://json-schema.org/draft-07/schema#',
     '$id': 'http://json-schema.org/draft-07/schema#',
     'title': 'Core schema meta-schema',
     'definitions': {'schemaArray': {'type': 'array',
                                     'minItems': 1,
                                     'items': {'$ref': '#'}},
                     'nonNegativeInteger': {'type': 'integer',
                                            'minimum': 0},
                     'nonNegativeIntegerDefault0': {'allOf': [{'$ref': '#/definitions/nonNegativeInteger'},
                                                              {'default': 0}]},
                     'simpleTypes': {'enum': ['array',
                                              'boolean',
                                            

Map (num_proc=6):  21%|██        | 908/4319 [00:00<00:00, 4304.41 examples/s]

...

parameter_schema error: 'The detailed properties expected when submitting a court filing.' is not of type 'object', 'boolean'

Failed validating 'type' in metaschema['properties']['properties']['additionalProperties']:
    {'$schema': 'http://json-schema.org/draft-07/schema#',
     '$id': 'http://json-schema.org/draft-07/schema#',
     'title': 'Core schema meta-schema',
     'definitions': {'schemaArray': {'type': 'array',
                                     'minItems': 1,
                                     'items': {'$ref': '#'}},
                     'nonNegativeInteger': {'type': 'integer',
                                            'minimum': 0},
                     'nonNegativeIntegerDefault0': {'allOf': [{'$ref': '#/definitions/nonNegativeInteger'},
                                                              {'default': 0}]},
                     'simpleTypes': {'enum': ['array',
                                              'boolean',
                                    

Map (num_proc=6):  96%|█████████▌| 4148/4319 [00:00<00:00, 4904.31 examples/s]

parameter_schema error: 'The schema for the parameters of the action.' is not of type 'object', 'boolean'

Failed validating 'type' in metaschema['properties']['properties']['additionalProperties']:
    {'$schema': 'http://json-schema.org/draft-07/schema#',
     '$id': 'http://json-schema.org/draft-07/schema#',
     'title': 'Core schema meta-schema',
     'definitions': {'schemaArray': {'type': 'array',
                                     'minItems': 1,
                                     'items': {'$ref': '#'}},
                     'nonNegativeInteger': {'type': 'integer',
                                            'minimum': 0},
                     'nonNegativeIntegerDefault0': {'allOf': [{'$ref': '#/definitions/nonNegativeInteger'},
                                                              {'default': 0}]},
                     'simpleTypes': {'enum': ['array',
                                              'boolean',
                                              'integer',

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_domain_actions_v1/commit/b319e693223362f72e180ced868d6d9c3fdc1930', commit_message='Upload dataset', commit_description='', oid='b319e693223362f72e180ced868d6d9c3fdc1930', pr_url=None, pr_revision=None, pr_num=None)